### Домашнє завдання: Пониження розмірностей для Аналізу Портретів Клієнтів

#### Контекст
В цьому ДЗ ми попрацюємо з методами пониження розмірності на наборі даних для задачі аналізу портретів клієнтів (Customer Personality Analysis). **В попередньому ДЗ ми працювали з цими даними використовуючи кластеризацію, зараз використаємо кластеризацію і візуалізауємо результати з різними методами.**

Customer Personality Analysis - це аналіз різних сегментів клієнтів компанії. Цей аналіз дозволяє бізнесу краще розуміти своїх клієнтів і полегшує процес адаптації продуктів під конкретні потреби, поведінку та інтереси різних типів клієнтів.

Аналіз портретів клієнтів допомагає бізнесу змінювати свій продукт на основі цільової аудиторії, розділеної на різні сегменти. Наприклад, замість того, щоб витрачати гроші на маркетинг нового продукту для всіх клієнтів у базі даних компанії, бізнес може проаналізувати, який сегмент клієнтів найімовірніше придбає продукт, і потім зосередити маркетингові зусилля лише на цьому сегменті.

#### Вхідні дані
Вам надано набір даних з такими атрибутами:

**Характеристики користувачів:**
- `ID`: Унікальний ідентифікатор клієнта
- `Year_Birth`: Рік народження клієнта
- `Education`: Рівень освіти клієнта
- `Marital_Status`: Сімейний стан клієнта
- `Income`: Річний дохід домогосподарства клієнта
- `Kidhome`: Кількість дітей у домогосподарстві клієнта
- `Teenhome`: Кількість підлітків у домогосподарстві клієнта
- `Dt_Customer`: Дата реєстрації клієнта у компанії
- `Recency`: Кількість днів з моменту останньої покупки клієнта
- `Complain`: 1, якщо клієнт скаржився за останні 2 роки, 0 - якщо ні

**Продукти:**
- `MntWines`: Сума, витрачена на вино за останні 2 роки
- `MntFruits`: Сума, витрачена на фрукти за останні 2 роки
- `MntMeatProducts`: Сума, витрачена на м'ясні продукти за останні 2 роки
- `MntFishProducts`: Сума, витрачена на рибні продукти за останні 2 роки
- `MntSweetProducts`: Сума, витрачена на солодощі за останні 2 роки
- `MntGoldProds`: Сума, витрачена на золото за останні 2 роки

**Акції:**
- `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
- `AcceptedCmp1`: 1, якщо клієнт прийняв пропозицію у першій кампанії, 0 - якщо ні
- `AcceptedCmp2`: 1, якщо клієнт прийняв пропозицію у другій кампанії, 0 - якщо ні
- `AcceptedCmp3`: 1, якщо клієнт прийняв пропозицію у третій кампанії, 0 - якщо ні
- `AcceptedCmp4`: 1, якщо клієнт прийняв пропозицію у четвертій кампанії, 0 - якщо ні
- `AcceptedCmp5`: 1, якщо клієнт прийняв пропозицію у п'ятій кампанії, 0 - якщо ні
- `Response`: 1, якщо клієнт прийняв пропозицію в останній кампанії, 0 - якщо ні

**Взаємодія з компанією:**
- `NumWebPurchases`: Кількість покупок, зроблених через вебсайт компанії
- `NumCatalogPurchases`: Кількість покупок, зроблених за каталогом
- `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
- `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць


Для початку, запустіть код нижче. Всі ці кроки ми робили в попередньому ДЗ і для того, щоб результати кластеризації у нас були схожими, потрібно аби передобробка була однаковою.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/marketing_campaign.csv', delimiter=',')

In [19]:
# Виводимо основну інформацію про дані
print("Кількість рядків і колонок:", df.shape)
print("\nТипи даних кожної колонки:\n", df.dtypes)
print("\nКількість пропущених значень у кожній колонці:\n", df.isnull().sum())

Кількість рядків і колонок: (2240, 29)

Типи даних кожної колонки:
 ID                       int64
Year_Birth               int64
Education               object
Marital_Status          object
Income                 float64
Kidhome                  int64
Teenhome                 int64
Dt_Customer             object
Recency                  int64
MntWines                 int64
MntFruits                int64
MntMeatProducts          int64
MntFishProducts          int64
MntSweetProducts         int64
MntGoldProds             int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumCatalogPurchases      int64
NumStorePurchases        int64
NumWebVisitsMonth        int64
AcceptedCmp3             int64
AcceptedCmp4             int64
AcceptedCmp5             int64
AcceptedCmp1             int64
AcceptedCmp2             int64
Complain                 int64
Z_CostContact            int64
Z_Revenue                int64
Response                 int64
dtype: object

Кількість пропущен

In [22]:
# 2. Обробка пропущених значень
df['Income_not_filled'] = df.Income.isna()
df.Income = df.Income.fillna(-1)

# 3. Обробка дати реєстрації
df.Dt_Customer = pd.to_datetime(df.Dt_Customer, format='%d/%m/%Y', dayfirst=True)
today = df.Dt_Customer.max()
df['days_lifetime'] = (today - df.Dt_Customer).dt.days
df['years_customer'] = df.Year_Birth.apply(lambda x: today.year - x)

# 4. Категоризація рівня освіти
df_education = pd.get_dummies(df.Education, prefix='education').astype(int)
df = pd.concat([df, df_education], axis=1)

# 5. Очищення сімейного стану
marital_status_map = {'Alone': 'Single', 'Absurd': 'Else', 'YOLO': 'Else'}
df['Marital_Status_clean'] = df.Marital_Status.map(marital_status_map)
df_ms = pd.get_dummies(df.Marital_Status_clean, prefix='marital').astype(int)
df = pd.concat([df, df_ms], axis=1)

# 6. Форматування доходу і видалення викиду
df.Income = df.Income.astype(int)
df = df[df.Income != 666666]

# 7. Створення фінального набору даних
X = df.drop(['ID', 'Dt_Customer', 'Education', 'Marital_Status', 'Marital_Status_clean'], axis=1)
X.reset_index(drop=True, inplace=True)

In [21]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Income_not_filled
0,5524,1957,Graduation,Single,58138.0,0,0,04/09/2012,58,635,...,0,0,0,0,0,0,3,11,1,False
1,2174,1954,Graduation,Single,46344.0,1,1,08/03/2014,38,11,...,0,0,0,0,0,0,3,11,0,False
2,4141,1965,Graduation,Together,71613.0,0,0,21/08/2013,26,426,...,0,0,0,0,0,0,3,11,0,False
3,6182,1984,Graduation,Together,26646.0,1,0,10/02/2014,26,11,...,0,0,0,0,0,0,3,11,0,False
4,5324,1981,PhD,Married,58293.0,1,0,19/01/2014,94,173,...,0,0,0,0,0,0,3,11,0,False


### Завдання 1: Виконання кластеризації та пониження розмірності для візуалізації результатів

Ваше завдання — провести кластеризацію клієнтів та візуалізувати результати кластеризації, використовуючи метод головних компонент (PCA) для пониження розмірності даних.

#### Інструкції:

1. **Вибір ключових характеристик:**
   Давайте обмежимось тільки наступними хараткеристиками для кластеризації цього разу:
   - `Income`: Річний дохід домогосподарства клієнта
   - `Recency`: Кількість днів з моменту останньої покупки клієнта
   - `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
   - `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
   - `days_lifetime`: Кількість днів з моменту реєстрації клієнта у компанії
   - `years_customer`: Вік клієнта
   - `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць
   Відберіть в наборі даних `X` лише ці характеристики.

2. **Нормалізація даних:**
   Використайте метод `MinMaxScaler` для нормалізації значень обраних характеристик.

3. **Кластеризація:**
   Проведіть кластеризацію клієнтів, використовуючи метод `KMeans` з трьома кластерами.

4. **Пониження розмірності:**
   Використайте метод головних компонент (PCA) для пониження розмірності даних до трьох компонент.

5. **Візуалізація результатів:**
   Використовуючи plolty express побудуйте 3D-графік розподілу клієнтів у просторі трьох головних компонент, де кольором позначено кластери.

6. **Опишіть, що спостерігаєте:**
   Чи кластеризація чітко розділила дані?

Далі ми детальніше проінтерпретуємо результати візуалізації і пониження розмірностей.

In [23]:
import pandas as pd

# Обираємо лише необхідні колонки
X = df[['Income', 'Recency', 'NumStorePurchases', 'NumDealsPurchases', 'days_lifetime', 'years_customer', 'NumWebVisitsMonth']]

In [24]:
from sklearn.preprocessing import MinMaxScaler

# Ініціалізуємо MinMaxScaler
scaler = MinMaxScaler()

# Нормалізуємо дані
X_scaled = scaler.fit_transform(X)

In [25]:
from sklearn.cluster import KMeans

# Ініціалізуємо KMeans з 3 кластерами
kmeans = KMeans(n_clusters=3, random_state=42)

# Виконуємо кластеризацію
clusters = kmeans.fit_predict(X_scaled)

# Додаємо отримані кластери в наш датафрейм
X['Cluster'] = clusters

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
<ipython-input-25-40fdf9628eee>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Cluster'] = clusters


In [26]:
from sklearn.decomposition import PCA

# Ініціалізуємо PCA з 3 компонентами
pca = PCA(n_components=3)

# Застосовуємо PCA до нормалізованих даних
X_pca = pca.fit_transform(X_scaled)

# Перетворюємо PCA компоненти в датафрейм для зручності
X_pca_df = pd.DataFrame(X_pca, columns=['PCA1', 'PCA2', 'PCA3'])
X_pca_df['Cluster'] = clusters

In [27]:
import plotly.express as px

# Створюємо 3D-графік
fig = px.scatter_3d(X_pca_df, x='PCA1', y='PCA2', z='PCA3', color='Cluster', title='Кластеризація клієнтів в просторі PCA')

# Відображаємо графік
fig.show()

Насправді дані досить чітко розподілені з одиничними викидами

### Завдання 2: Аналіз результатів пониження розмірності

1. **Розрахунок частки поясненої дисперсії:**
   Визначте, яка частка загальної варіації даних пояснюється кожною з трьох головних компонент (PC1, PC2, PC3) за допомогою атрибуту `explained_variance_ratio_` об'єкта PCA. Виведіть результат на екран.

2. **Розрахунок кумулятивної частки поясненої дисперсії:**
   Обчисліть кумулятивну частку поясненої дисперсії для трьох головних компонент, щоб зрозуміти, скільки варіації даних пояснюється першими кількома компонентами.

In [30]:
# Частка поясненої дисперсії для кожної компоненти
explained_variance_ratio = pca.explained_variance_ratio_

# Виводимо частку поясненої дисперсії для кожної компоненти
for i, variance in enumerate(explained_variance_ratio):
    print(f"Частка поясненої дисперсії для компоненти PC{i+1}: {variance:.4f}")

# Обчислюємо кумулятивну частку поясненої дисперсії
cumulative_explained_variance = explained_variance_ratio.cumsum()

# Виводимо кумулятивну частку поясненої дисперсії
for i, cumulative_variance in enumerate(cumulative_explained_variance):
    print(f"Кумулятивна частка поясненої дисперсії до компоненти PC{i+1}: {cumulative_variance:.4f}")

Частка поясненої дисперсії для компоненти PC1: 0.3020
Частка поясненої дисперсії для компоненти PC2: 0.2867
Частка поясненої дисперсії для компоненти PC3: 0.2512
Кумулятивна частка поясненої дисперсії до компоненти PC1: 0.3020
Кумулятивна частка поясненої дисперсії до компоненти PC2: 0.5887
Кумулятивна частка поясненої дисперсії до компоненти PC3: 0.8399


Перші три компоненти (PC1, PC2, PC3) разом пояснюють 83.99% варіації в даних. Це дуже хороший результат для PCA, оскільки ми значно знизили розмірність (від початкової кількості характеристик до трьох компонент), зберігши при цьому більшість інформації.
PC1 пояснює найбільшу частку варіації (30.20%), але наступні компоненти (PC2 та PC3) також важливі, оскільки значно доповнюють загальну частку поясненої дисперсії.
Залишкові 16.01% варіації, не пояснені цими трьома компонентами, можуть містити менш значну або шумову інформацію.
Цей аналіз підтверджує, що PCA ефективно зменшив розмірність, зберігаючи при цьому більшість корисної інформації з даних.

### Завдання 3: Інтерпретація "Loadings"

Продовжуємо інтерпретацію результатів `PCA`і познайомимось з новим поняттям `loadings`, яке допоможе нам знайти звʼязок між головними компонентами і оригінальними ознаками в наборі даних.

Ми зараз побудували візуалізацію кластерів точок даних в просторі трьох головних компонент. Але хочеться знайти звʼязок між головними компонентами і оригінальними ознаками. Для розуміння, які початкові характеристики даних мають найбільший вплив на ці головні компоненти, ми можемо використати атрибут `components_` методу `PCA`.

#### Що таке `pca.components_`?

`pca.components_` — це масив, який містить коефіцієнти (або "ваги"), що показують внесок кожної вихідної ознаки у кожну з головних компонент. Ці коефіцієнти ще називаються **"loading"** або "навантаженнями" компонент.

- **Loadings** (`навантаження`) відображають важливість кожної змінної (ознаки) для відповідної головної компоненти. Вони показують, яким чином змінні поєднуються, щоб утворити нові, зменшені вимірювання.
- Якщо коефіцієнт має високе абсолютне значення (як позитивне, так і негативне), це вказує на те, що відповідна змінна сильно впливає на головну компоненту.

#### Саме завдання
Ваше завдання — обчислити "навантаження" для кожної з головних компонент і інтерпретувати результати.

1. **Обчислення loadings для компонент:**
   Використайте атрибут `components_` об'єкта PCA для створення DataFrame, який відображатиме внесок кожної вихідної ознаки в кожну головну компоненту.

2. **Інтерпретація результатів:**
   Виведіть значення "навантажень" і проаналізуйте, які ознаки найбільше впливають на кожну головну компоненту.

In [33]:
# Виключаємо стовпець 'Cluster' перед створенням DataFrame з loadings
X_without_cluster = X.drop(columns=['Cluster'])

# Створюємо DataFrame для відображення "навантажень"
loadings = pd.DataFrame(pca.components_, columns=X_without_cluster.columns)

# Додаємо індекси для компонент
loadings.index = [f'PC{i+1}' for i in range(pca.n_components_)]

# Виводимо результат
print(loadings)

       Income   Recency  NumStorePurchases  NumDealsPurchases  days_lifetime  \
PC1  0.063557  0.475786           0.284282           0.103049       0.821912   
PC2  0.047082 -0.878876           0.187195           0.059324       0.431061   
PC3 -0.373826 -0.029910          -0.816668           0.050687       0.305490   

     years_customer  NumWebVisitsMonth  
PC1        0.012658           0.049530  
PC2       -0.001022           0.031377  
PC3       -0.080411           0.300089  


PC1 (Перша головна компонента):
Income: 0.063557
Вплив річного доходу на PC1 невеликий, але позитивний.
Recency: 0.475786
Вплив кількості днів з моменту останньої покупки на PC1 помірний і позитивний.
NumStorePurchases: 0.284282
Вплив кількості покупок у магазині помірний і позитивний.
NumDealsPurchases: 0.103049
Вплив кількості покупок зі знижками невеликий і позитивний.
days_lifetime: 0.821912
Великий позитивний вплив кількості днів з моменту реєстрації клієнта.
years_customer: 0.012658
Вплив віку клієнта на PC1 дуже маленький і позитивний.
NumWebVisitsMonth: 0.049530
Вплив кількості відвідувань вебсайту також невеликий і позитивний.
Інтерпретація: PC1 найбільше визначається значенням days_lifetime, яке має найбільше позитивне навантаження. Це свідчить про те, що ця компонента може вказувати на загальний час, протягом якого клієнт є зареєстрованим у компанії, з меншою увагою до інших ознак.

PC2 (Друга головна компонента):
Income: 0.047082
Невеликий позитивний вплив доходу на PC2.
Recency: -0.878876
Великий негативний вплив кількості днів з моменту останньої покупки. Це вказує на те, що ця компонента переважно визначається тим, наскільки нещодавно клієнт робив покупку.
NumStorePurchases: 0.187195
Невеликий позитивний вплив кількості покупок у магазині.
NumDealsPurchases: 0.059324
Невеликий позитивний вплив кількості покупок зі знижками.
days_lifetime: 0.431061
Помірний позитивний вплив кількості днів з моменту реєстрації.
years_customer: -0.001022
Дуже маленький негативний вплив віку клієнта.
NumWebVisitsMonth: 0.031377
Невеликий позитивний вплив кількості відвідувань вебсайту.
Інтерпретація: PC2 в основному визначається Recency, тобто, ця компонента відображає те, наскільки недавно клієнт здійснив покупку. Це може бути корисним для аналізу активності клієнтів.

PC3 (Третя головна компонента):
Income: -0.373826
Помірний негативний вплив доходу на PC3.
Recency: -0.029910
Дуже маленький негативний вплив кількості днів з моменту останньої покупки.
NumStorePurchases: -0.816668
Великий негативний вплив кількості покупок у магазині.
NumDealsPurchases: 0.050687
Невеликий позитивний вплив кількості покупок зі знижками.
days_lifetime: 0.305490
Помірний позитивний вплив кількості днів з моменту реєстрації.
years_customer: -0.080411
Невеликий негативний вплив віку клієнта.
NumWebVisitsMonth: 0.300089
Помірний позитивний вплив кількості відвідувань вебсайту.

###Завдання 4
Давайте проаналізуємо "навантаження" (**loadings**) для трьох головних компонент після вилучення ознаки `Income`. Це допоможе нам зрозуміти, як змінилася важливість інших ознак для кожної головної компоненти, коли одна з ключових характеристик (`Income`) була вилучена.

#### Кроки для проведення аналізу і ваше завдання:

1. Видаліть ознаку `Income` з нашого набору даних `X` і повторно виконайте PCA (метод головних компонент) для отримання нових "навантажень".

2. Обчисліть нові "навантаження" для трьох головних компонент на наборі даних без `Income`

3. Проаналізуйте, які ознаки мають найбільший вплив на кожну головну компоненту після вилучення `Income`.

4. Перегляньте, наскільки кожна з головних компонент пояснює дисперсію в даних без ознаки `Income`.

In [34]:
# Видалення ознаки Income
X_no_income = X.drop(columns=['Income'])

In [35]:
from sklearn.decomposition import PCA

# Ініціалізація PCA
pca_no_income = PCA(n_components=3)

# Підгонка PCA до даних без Income
pca_no_income.fit(X_no_income)

PCA(n_components=3)

In [36]:
# Отримання нових "навантажень"
loadings_no_income = pd.DataFrame(pca_no_income.components_, columns=X_no_income.columns)

# Додаємо індекси для компонент
loadings_no_income.index = [f'PC{i+1}' for i in range(pca_no_income.n_components_)]

# Виведення нових навантажень
print(loadings_no_income)

      Recency  NumStorePurchases  NumDealsPurchases  days_lifetime  \
PC1 -0.003598          -0.001780          -0.002089      -0.999979   
PC2  0.999935          -0.000191          -0.000403      -0.003584   
PC3 -0.010118           0.038810           0.010756       0.001352   

     years_customer  NumWebVisitsMonth   Cluster  
PC1        0.001331          -0.003271  0.002892  
PC2        0.010083          -0.002418 -0.003013  
PC3        0.998825          -0.024908 -0.001131  


In [37]:
# Частка поясненої дисперсії для нових компонент
explained_variance_ratio_no_income = pca_no_income.explained_variance_ratio_

# Кумулятивна частка поясненої дисперсії
cumulative_variance_no_income = explained_variance_ratio_no_income.cumsum()

# Виведення результатів
print("Частка поясненої дисперсії для нових компонент:")
for i, variance in enumerate(explained_variance_ratio_no_income):
    print(f"PC{i+1}: {variance:.4f}")

print("\nКумулятивна частка поясненої дисперсії до кожної компоненти:")
for i, cumulative_variance in enumerate(cumulative_variance_no_income):
    print(f"PC{i+1}: {cumulative_variance:.4f}")

Частка поясненої дисперсії для нових компонент:
PC1: 0.9761
PC2: 0.0200
PC3: 0.0034

Кумулятивна частка поясненої дисперсії до кожної компоненти:
PC1: 0.9761
PC2: 0.9961
PC3: 0.9995


Видалення ознаки Income значно вплинуло на структуру головних компонент, показавши, що інші ознаки, такі як days_lifetime та years_customer, стали більш важливими для пояснення дисперсії даних. Важливо врахувати ці результати при подальшому аналізі та прийнятті рішень на основі даних.

### Завдання 5: Візуалізація кластеризації за допомогою t-SNE

Ваше завдання — використати метод t-SNE для візуалізації результатів кластеризації клієнтів у двовимірному просторі. Метод t-SNE допомагає знизити розмірність даних та зберегти локальні структури в даних, що робить його ефективним для візуалізації високорозмірних даних. Ми також зможемо порівняти результат цього методу з РСА.

1. Використайте метод t-SNE для зниження розмірності до 2х вимірів даних, які включають ознаки всі, що і в завданні 1, а також були відмасштабовані перед пониженням розмірностей.

2. Створіть новий DataFrame з координатами, отриманими після застосування t-SNE, та додайте до нього мітки кластерів.

3. Побудуйте інтерактивний 2D-графік розподілу клієнтів, де кольором буде позначено різні кластери і проаналізуйте графік з рекомендаціями нижче (можливо треба буде вивести додаткові візуалізації чи таблиці для інтерпретації, але треба прям зрозуміти, які ознаки формують який кластер і чим кластери відрізняються одне від одного).

  **Опишіть отримані кластери з точки зору ознак.**

4. Опишіть відмінність графіка tSNE від PCA.

#### ЯК можна інтерпретувати з t-SNE?

Хоча t-SNE не надає "компонентів" як РСА, він забезпечує низьковимірне представлення даних, яке можна візуально інтерпретувати:

- **Кластери:** t-SNE особливо добре показує кластери подібних точок. Якщо ви бачите чітко визначені кластери на графіку t-SNE, це свідчить про наявність груп схожих спостережень у ваших даних. Проаналізувати їх можемо, якщо додамо дані в `hover_data` або якщо якісь з даних виведемо як розмір чи форма точок на візуалізації. Також корисно може бути вивести середні значення ознак по кластерам.
- **Локальна структура:** Відносне розташування точок одного кластеру на графіку t-SNE може допомогти вам зрозуміти, які дані подібні між собою.
- **Глобальна структура:** Будьте обережні; t-SNE менш надійний для відображення глобальних структур (наприклад, відстаней між кластерами) у порівнянні з PCA, бо t-SNE націлений на збереження саме локальних структур.

In [38]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import plotly.express as px

# Припустимо, що `X` — це наш датафрейм без ознаки `Income` і вже нормалізований.
# Включаємо нормалізацію перед застосуванням t-SNE.
features = ['Recency', 'NumStorePurchases', 'NumDealsPurchases', 'days_lifetime', 'years_customer', 'NumWebVisitsMonth']

# Нормалізуємо дані
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X[features])

# Застосовуємо t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_scaled)

# Створюємо DataFrame з результатами t-SNE
df_tsne = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2'])
df_tsne['Cluster'] = X['Cluster']

# Візуалізуємо результати
fig = px.scatter(df_tsne, x='TSNE1', y='TSNE2', color='Cluster',
                 title='t-SNE Clustering Visualization',
                 labels={'TSNE1': 't-SNE Component 1', 'TSNE2': 't-SNE Component 2'},
                 color_continuous_scale=px.colors.qualitative.Set1)

fig.show()

In [39]:
import pandas as pd

# Припустимо, що X містить всі ознаки разом з мітками кластерів.
# Групуємо дані за кластерами та обчислюємо середні значення для кожної ознаки
cluster_means = X.groupby('Cluster').mean()

# Виводимо результати
print(cluster_means)

               Income    Recency  NumStorePurchases  NumDealsPurchases  \
Cluster                                                                  
0        52835.119048  36.483193           6.726891           3.012605   
1        50349.671951  79.179268           5.362195           2.100000   
2        51208.812766  26.958865           5.343262           1.887943   

         days_lifetime  years_customer  NumWebVisitsMonth  
Cluster                                                    
0           553.358543       45.088235           5.876751  
1           328.232927       45.441463           5.239024  
2           180.685106       45.025532           4.838298  


На мою думку кластери менш чітко відокремлені для t-SNE, можливо, потрібно налаштувати параметри t-SNE. Є багато перетинів і більше викидів. Дані кластеру 0 та 1 ймовірно часто подібні між собою.

Кластер 0 включає клієнтів з високою активністю в покупках, як у магазинах, так і онлайн, а також з вищим числом днів з моменту реєстрації.

Кластер 1 містить менш активних клієнтів, з найвищим показником Recency та меншим числом покупок зі знижками.

Кластер 2 містить дуже активних клієнтів з найнижчим показником Recency і меншим числом днів з моменту реєстрації.

Кластер 0 має більшу схильність до покупок зі знижками, що може свідчити про чутливість до знижок або акцій.
Онлайн активність:

Клієнти в Кластері 0 мають вищу активність на вебсайті, що може бути корисним для планування онлайн-маркетингових кампаній.